In [ ]:
!pip install -q scikit-learn sentence-transformers transformers torch numpy accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 12.4 MB/s eta 0:00:00


In [ ]:
from typing import List, Dict
import numpy as np
import torch
from sklearn.ensemble import RandomForestRegressor
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
class WeatherAnalysisAgent:
    def __init__(self):
        self.model = RandomForestRegressor(n_estimators=100)

    def train(self, historical_data: Dict):
        X = np.array([[d['month'], d['latitude'], d['longitude']] for d in historical_data])
        y = np.array([d['weather_score'] for d in historical_data])
        self.model.fit(X, y)

    def predict_best_time(self, location: Dict) -> Dict:
        predictions = []
        for month in range(1, 13):
            prediction = self.model.predict([[
                month,
                location['latitude'],
                location['longitude']
            ]]).item()
            predictions.append({'month': month, 'score': float(prediction)})

        return {
            'best_months': sorted(predictions, key=lambda x: x['score'], reverse=True)[:3],
            'location': location
        }

In [ ]:
class HotelRecommenderAgent:
    def __init__(self):
        self.encoder = SentenceTransformer('all-MiniLM-L6-v2')
        self.hotels_db = []
        self.hotels_embeddings = None

    def add_hotels(self, hotels: List[Dict]):
        self.hotels_db = hotels
        descriptions = [h['description'] for h in hotels]
        self.hotels_embeddings = self.encoder.encode(descriptions)

    def find_hotels(self, preferences: str, top_k: int = 5) -> List[Dict]:
        pref_embedding = self.encoder.encode([preferences])
        similarities = np.dot(self.hotels_embeddings, pref_embedding.T).flatten()
        top_indices = similarities.argsort()[-top_k:][::-1]
        return [
            {**self.hotels_db[i], 'similarity_score': float(similarities[i])}
            for i in top_indices
        ]

In [ ]:
class ItineraryPlannerAgent:
    def __init__(self):
        print("Qwen2.5-3B 모델 로딩 중...")

        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4"
        )

        model_name = "Qwen/Qwen2.5-3B-Instruct"

        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quantization_config,
            device_map="auto",
            trust_remote_code=True
        )

        print("모델 로딩 완료!")

    def create_itinerary(self, destination_info: Dict, weather_info: Dict,
                         hotel_info: Dict, duration: int) -> Dict:
        prompt = self._create_prompt(destination_info, weather_info, hotel_info, duration)

        messages = [
            {"role": "system", "content": "당신은 전문 여행 플래너입니다. 구체적이고 실용적인 여행 일정을 작성해주세요."},
            {"role": "user", "content": prompt}
        ]

        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        model_inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)

        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens=800,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )

        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        return {
            'itinerary': response,
            'duration': duration,
            'destination': destination_info['name']
        }

    def _create_prompt(self, destination_info: Dict, weather_info: Dict,
                       hotel_info: Dict, duration: int) -> str:
        return f"""다음 정보를 바탕으로 {duration}일간의 상세한 여행 일정을 작성해주세요:

목적지: {destination_info['name']}
최적 방문 시기: {weather_info['best_months'][0]['month']}월
숙박: {hotel_info[0]['name']}
주요 관광지: {', '.join(destination_info['attractions'])}

각 날짜별로 아침, 점심, 저녁 일정을 포함하여 구체적으로 작성해주세요."""

In [ ]:
class SummaryAgent:
    def __init__(self):
        print("Qwen2.5-3B 모델 로딩 중... (SummaryAgent)")

        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4"
        )

        model_name = "Qwen/Qwen2.5-3B-Instruct"

        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quantization_config,
            device_map="auto",
            trust_remote_code=True
        )

        print("모델 로딩 완료!")

    def calculate_total_price(self, hotel_info: Dict, duration: int) -> float:
        hotel_cost = hotel_info[0]['price'] * duration
        daily_expenses = 100
        return hotel_cost + daily_expenses * duration

    def create_email(self, trip_data: Dict, client_name: str) -> Dict:
        total_price = self.calculate_total_price(
            trip_data['recommended_hotels'],
            trip_data['itinerary']['duration']
        )

        prompt = f"""다음 여행 계획 정보를 바탕으로 고객에게 보낼 전문적인 이메일을 작성해주세요:

고객명: {client_name}
목적지: {trip_data['itinerary']['destination']}
기간: {trip_data['itinerary']['duration']}일
최적 방문 시기: {trip_data['weather_analysis']['best_months'][0]['month']}월
추천 호텔: {trip_data['recommended_hotels'][0]['name']}
예상 총 비용: ${total_price}

여행 일정:
{trip_data['itinerary']['itinerary']}

정중하고 전문적인 톤으로 작성해주세요."""

        messages = [
            {"role": "system", "content": "당신은 전문 여행사 직원입니다. 고객에게 보낼 정중하고 상세한 이메일을 작성해주세요."},
            {"role": "user", "content": prompt}
        ]

        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        model_inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)

        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens=1000,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )

        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        return {
            'email_content': response,
            'total_price': total_price
        }

In [ ]:
class TravelPlanningSystem:
    def __init__(self):
        self.weather_agent = WeatherAnalysisAgent()
        self.hotel_agent = HotelRecommenderAgent()
        self.itinerary_agent = ItineraryPlannerAgent()
        self.summary_agent = SummaryAgent()

    def setup(self, weather_data, hotel_data):
        self.weather_agent.train(weather_data)
        self.hotel_agent.add_hotels(hotel_data)

    def plan_trip(self, destination, preferences, duration, client_name):
        weather = self.weather_agent.predict_best_time(destination)
        hotels = self.hotel_agent.find_hotels(preferences)
        itinerary = self.itinerary_agent.create_itinerary(destination, weather, hotels, duration)
        summary = self.summary_agent.create_email({
            'weather_analysis': weather,
            'recommended_hotels': hotels,
            'itinerary': itinerary
        }, client_name)
        return summary

# Sample usage
historical_weather_data = [{'month': m, 'latitude': 41.9, 'longitude': 12.5, 'weather_score': 0.5 + 0.05 * m} for m in range(1,13)]
hotels_database = [
    {'name': 'Grand Hotel', 'description': 'Luxury hotel in city center with spa and restaurant', 'price': 300},
    {'name': 'Boutique Resort', 'description': 'Boutique hotel with personalized service', 'price': 250}
]
destination = {'name': 'Rome', 'latitude': 41.9, 'longitude': 12.5, 'attractions': ['Colosseum', 'Vatican']}
preferences = "Luxury hotel with spa and restaurant"
system = TravelPlanningSystem()
system.setup(historical_weather_data, hotels_database)
result = system.plan_trip(destination, preferences, 3, "John Smith")
print(result['email_content'])
print("Estimated Total Cost: $", result['total_price'])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

모델 로딩 완료!
Qwen2.5-3B 모델 로딩 중... (SummaryAgent)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

모델 로딩 완료!
안녕하세요, 존 스미스 씨,

첫눈에 반했답니다, 당신이 제안하신 Rome 여행 일정! 저희와 함께하는 이번 여행을 기대하며, 그 동안 필요한 정보나 조언을 드리기 위해 이 메일을 보내왔습니다.

Rome에서의 3일간의 여행을 시작하는 날, 당신의 첫 방문은 바로 Grand Hotel에서 이루어질 것입니다. 그곳에서 아침 식사를 즐길 수 있으며, 편안하게 잠들 수 있을 거라 믿습니다. 그리고 그날 오후는 Roman Forum과 사원을 방문하는 것으로, 로마의 역사를 배울 수 있을 것입니다. 저녁에는 Piazza Navona에서 편안하게 쉬면서 감상해보실 수 있겠죠. 

두 번째 날은 Vatican City를 방문하며, 그곳에서 Raphael Rooms, Sistine Chapel, 그리고 Gallery of Maps를 방문하실 수 있습니다. 이 곳에서의 시간은 매우 특별하며, 이 곳을 빠져서는 안 되는 명소 중 하나라고 생각합니다. 그리고 이날 밤에는 Trastevere라는 아름다운 지역의 맛집에서 저녁 식사를 즐길 수 있을 것입니다.

마지막 날은 Rome의 특별한 순간을 만들기 위해, Vatican City에서 더 많은 시간을 보내거나, 나머지 시간을 이용해서 다른 명소들을 방문하실 수 있겠죠. 또한, 저녁 식사 후에는 Grand Hotel에서 마지막 저녁을 즐길 수 있을 것입니다.

Rome의 날씨는 추워질 수 있으니, 가방을 미리 준비해 주셨으면 합니다. 날씨가 좋은 날에는 호텔 근처의 공원에서 풍경을 감상해 보실 수 있을 것 같습니다. Rome에는 많은 볼거리를 있으니, 이 일정 외에도 방문할 수 있는 곳들도 찾아보셨으면 합니다.

여행을 시작하려는 당신의 마음을 환영합니다. Rome에서의 시간을 즐길 수 있도록 최선을 다하겠습니다. 궁금한 점이나 도움이 필요하시면 언제든지 알려주세요.

감사합니다.

Warmest regards,

[Your Name]
[Your Company Name]
Estimated Total Cost: $ 